In [ ]:
import csv
import cv2
import numpy as np

lines = []

# loads in the recorded training data which correlates captured images along with telemetry data
# this section loads in each line from the CSV file
with open('../../udacity/behavioral_data/my_data_gold/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)
        
images = []
measurements = []

for line in lines:
    # for each line in the telemetry data, open the image captured from the center angle
    # append the image and also append a copy of the image that his been flipped across the horizontal axis
    # this both increases the amount of training/validation data and should help the model generalize
    image = cv2.imread(line[0])
    images.append(image)
    images.append(np.fliplr(image))      
    
    # for each line in the telemetry data, also want to grab the steering angle information
    # this is the 4th item in each line, and it is grabbed accordingly
    # since we add the flipped copy of the image to the image array, we multiply a copy of the
    # steering angle multiplied by -1.0 and append both the original and inverted value to the 
    # measurements list
    measurement = float(line[3])
    measurements.append(measurement)
    measurements.append(-1.0 * measurement)

# with the image and measurement lists created from the training data, we want to make them numpy arrays
X_train = np.array(images)
y_train = np.array(measurements)

In [ ]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Convolution2D, MaxPooling2D, Dropout, Activation, Cropping2D 

# nVidia arch, from project instruction video
model = Sequential()

# normalize the input data
model.add(Lambda(lambda x: x/255.0 - 0.5, input_shape=(160,320,3)))

# crop the image to eliminate the sky region of each frame and the hood region as well
model.add(Cropping2D(cropping=((70,20), (0,0))))

# conv2d layer with filter size 5x5 and filter depth of 12, stride of 2x2; ReLU activation for nonlinearity
# dropout layer with 25% omission rate to combat overfitting
model.add(Convolution2D(12, 5, 5, subsample=(2,2), activation="relu"))
model.add(Dropout(0.25))

# conv2d layer with filter size 5x5 and filter depth of 18, stride of 2x2; ReLU activation for nonlinearity
# dropout layer with 25% omission rate to combat overfitting
model.add(Convolution2D(18, 5, 5, subsample=(2,2), activation="relu"))
model.add(Dropout(0.25))

# conv2d layer with filter size 5x5 and filter depth of 24, stride of 2x2; ReLU activation for nonlinearity
# dropout layer with 25% omission rate to combat overfitting
model.add(Convolution2D(24, 5, 5, subsample=(2,2), activation="relu"))
model.add(Dropout(0.25))

# conv2d layer with filter size 3x3 and filter depth of 32, stride of 1x1; ReLU activation for nonlinearity
# dropout layer with 25% omission rate to combat overfitting
model.add(Convolution2D(32, 3, 3, activation="relu"))

# flatten for fully connected layers
model.add(Flatten())

# 3 fully connected layers of size 30, 10, and finally 1 since we are only predicting steering angle as the output
model.add(Dense(30))
model.add(Dense(10))
model.add(Dense(1))

# loss function of MSE used with ADAM optimizer.
model.compile(loss='mse', optimizer='adam')

# 30% of the testing data is used for validation, data is shuffled, 10 epochs are used
model.fit(X_train, y_train, validation_split=0.3, shuffle=True, nb_epoch=10)

# save the trained model
model.save('model.h5')

print ("Model saved!")

In [ ]:
# Gold config so far: data from my_data_gold, using nVidiaNet, 1 epoch
# keep trying to add more training data and my model doesn't improve.
# I wonder if my the gold model got lucky... even redoing training with the same data doesn't seem to
# perform as well.

# adding dropout (25% loss) to the subsampled conv layers didn't do the model any favors
# adding dropout (10% loss) to only first subsampled conv layer didn't do much either
# adding dropout (25% loss) to the second to last conv layer (non subsampled) ...
# the unfortunate thing is the training and validation loss both look promising despite model 
# architecture changes and yet the car will still not perform well in testing
# a given model seems to learn the first major curve well but will consistently struggle after the 
# bridge on the next major corner

# removing one of the dense layers and the last conv non-subsampled layer did pretty well except
# for the turn after the bridge.
# 24_5_5_2x2ss_36_5_5_2x2ss_48_5_5_2x2ss_64_3_3_noss_50_10_1

# I thought turning off shuffling would perform really well, possibly overleveraging sequential 
# training samples to overfit to the test track. In reality, it performed worse.

# I tried increasing the validation split and moving to a second epoch and the model makes it around!
# better than the previous gold model

# NEW GOLD on my_data_gold
#model = Sequential()
#model.add(Lambda(lambda x: x/255.0 - 0.5, input_shape=(160,320,3)))
#model.add(Cropping2D(cropping=((70,20), (0,0))))
#model.add(Convolution2D(24, 5, 5, subsample=(2,2), activation="relu"))
#model.add(Convolution2D(36, 5, 5, subsample=(2,2), activation="relu"))
#model.add(Convolution2D(48, 5, 5, subsample=(2,2), activation="relu"))
#odel.add(Convolution2D(64, 3, 3, activation="relu"))
#model.add(Flatten())
#model.add(Dense(50))
#model.add(Dense(10))
#model.add(Dense(1))

# Another GOLD, on my_data_gold
#model = Sequential()
#model.add(Lambda(lambda x: x/255.0 - 0.5, input_shape=(160,320,3)))
#model.add(Cropping2D(cropping=((70,20), (0,0))))
#model.add(Convolution2D(12, 5, 5, subsample=(2,2), activation="relu"))
#model.add(Dropout(0.25))
#model.add(Convolution2D(18, 5, 5, subsample=(2,2), activation="relu"))
#model.add(Dropout(0.25))
#model.add(Convolution2D(24, 5, 5, subsample=(2,2), activation="relu"))
#model.add(Dropout(0.25))
#model.add(Convolution2D(32, 3, 3, activation="relu"))
#model.add(Flatten())
#model.add(Dense(30))
#model.add(Dense(10))
#model.add(Dense(1))

#introduction of dropout, fewer filters per layer, less connections in fully connected layers
# 30% validation split, 10 epochs